In [ ]:
from __future__ import print_function

%matplotlib inline
import numpy
import matplotlib.pyplot as plt
from scipy import integrate

import csv
import time

In [ ]:
def readApprox(filename):
    fp = open(filename,"r")
    csvFile = csv.reader(fp,delimiter=",")
    
    firstRow = next(csvFile)
    x = numpy.array(firstRow,dtype=numpy.float64)
    
    numRows = 0
    allData = []
    approx = numpy.arange(0)
    for line in csvFile:
        line = numpy.array(line,dtype=numpy.float64)
        allData.append(line)
        numRows += 1

    approx = numpy.zeros((numRows,len(allData[0])-1))
    timeSteps = numpy.zeros(numRows)
    numRows = 0
    for line in allData:
        approx[numRows,:] = line[1:]
        timeSteps[numRows] = line[0]
        numRows += 1
            
    fp.close()
    
    return(x,timeSteps,approx)
    


In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 1
%matplotlib notebook

In [ ]:
%autoreload 1
%matplotlib notebook

[x,timeSteps,approx] = readApprox("../build-butterflyVWaspPDE-Desktop-Debug/approximation.csv")
print("Final Time: {0:.3f}".format(timeSteps[-1]))
t    = 0
row  = 0
skip = 4;

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
plt.ion()
#fig.show()
for timeIndex in range(0,approx.shape[0],skip):
    #print(butterFly)
    
    axes.clear()
    axes.set_xlabel("x")
    axes.set_ylabel("Butterflies")
    axes.set_title("Butterflies at t={0:.5f}".format(timeSteps[timeIndex]))
    #axes.ticklabel_format(axis='y', style='sci', scilimits=(-1,1))
    axes.set_xlim(-1.1, 1.1)
    axes.set_ylim(-0.1, 1.1)

    butterflyPlot = axes.plot(x,approx[timeIndex],'r')
    fig.canvas.draw()
    time.sleep(0.0)





In [ ]:
%matplotlib inline
%autoreload 0
fig = plt.figure()
axes = fig.add_subplot(2, 1, 2)
[XGrid,TGrid] = numpy.meshgrid(x,timeSteps)
cp = axes.contourf(XGrid,TGrid,approx)
fig.colorbar(cp) # Add a colorbar to a plot
axes.set_title('Butterflies in time')
axes.set_xlabel(r'$\theta$')
axes.set_ylabel('Time')
plt.show()
